In [2]:
import torch as th
import numpy as np

## Tensor初始化方法

### 直接产生

In [3]:
data = [[1, 2],[3, 4]]
x_data = th.tensor(data) #生成以一个和之前形状一样的tensor
print(x_data)

tensor([[1, 2],
        [3, 4]])


### 从numpy中产生

In [5]:
data = [[1, 2],[3, 4]]
np_array = np.array(data)
x_np = th.from_numpy(np_array)
print(x_np)

tensor([[1, 2],
        [3, 4]])


### like初始化

In [6]:
x_ones = th.ones_like(x_data) # retains the properties of x_data
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = th.rand_like(x_data, dtype=th.float) # overrides the datatype of x_data
print(f"Random Tensor: \n {x_rand} \n")

Ones Tensor: 
 tensor([[1, 1],
        [1, 1]]) 

Random Tensor: 
 tensor([[0.1159, 0.9624],
        [0.8620, 0.1864]]) 



### 产生特定形状的tensor

In [7]:
shape = (2,3)
one_data = th.ones(shape)
rand_data = th.rand(shape)
zero_data = th.zeros(shape)
randint_data = th.randint(3, 10, shape)
print(f"Random Tensor: \n {randint_data} \n")
print(f"one tensor: {one_data}")
print(f"zero tensor: {zero_data}")
print(f"randint tensor: {randint_data}")

Random Tensor: 
 tensor([[5, 5, 6],
        [9, 3, 4]]) 

one tensor: tensor([[1., 1., 1.],
        [1., 1., 1.]])
zero tensor: tensor([[0., 0., 0.],
        [0., 0., 0.]])
randint tensor: tensor([[5, 5, 6],
        [9, 3, 4]])


## Tensor属性

In [8]:
tensor = th.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


## Tensor运算

### 判断cuda是否可用

In [9]:
if th.cuda.is_available():
    x_data = x_data.to("cuda")
    device = "cuda"

### 切片和索引操作

In [10]:
tensor = th.ones(4, 4)
print('First row: ',tensor[0])
print('First column: ', tensor[:, 0])
print('Last column:', tensor[..., -1])
tensor[:,1] = 0
print(tensor)

First row:  tensor([1., 1., 1., 1.])
First column:  tensor([1., 1., 1., 1.])
Last column: tensor([1., 1., 1., 1.])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


### 拼接操作

In [13]:
x_ten= th.rand((2,3))
t1 = th.cat([x_ten, x_ten, x_ten], dim=1) #dim = 1表示增加列
print(t1) 
t2 = th.cat([x_ten, x_ten, x_ten], dim=0) #dim = 0表示增加行
print(t2) 

tensor([[0.6564, 0.3021, 0.7075, 0.6564, 0.3021, 0.7075, 0.6564, 0.3021, 0.7075],
        [0.2535, 0.4922, 0.4656, 0.2535, 0.4922, 0.4656, 0.2535, 0.4922, 0.4656]])
tensor([[0.6564, 0.3021, 0.7075],
        [0.2535, 0.4922, 0.4656],
        [0.6564, 0.3021, 0.7075],
        [0.2535, 0.4922, 0.4656],
        [0.6564, 0.3021, 0.7075],
        [0.2535, 0.4922, 0.4656]])


### 算术运算
 

In [15]:
tensor = th.rand(2, 3)
y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T) #表示矩阵乘法
print(y1)
print(y2)

z1 = tensor * tensor
z2 = tensor.mul(tensor) #表示矩阵中按照element-wise进行乘法运算
print(z1)
print(z2)

tensor([[1.3707, 1.1670],
        [1.1670, 1.0958]])
tensor([[1.3707, 1.1670],
        [1.1670, 1.0958]])
tensor([[0.2970, 0.5838, 0.4900],
        [0.1786, 0.7748, 0.1424]])
tensor([[0.2970, 0.5838, 0.4900],
        [0.1786, 0.7748, 0.1424]])


### 运算元素查询

In [16]:
agg = tensor.sum()
agg_item = agg.item()
print(agg_item, type(agg_item))

3.6891863346099854 <class 'float'>


### 元素替换

In [19]:
tensor = th.rand((2,3))
print(tensor, "\n")
tensor.add_(5)
print(tensor)
tensor_y = th.rand((2,3))
tensor_y.copy_(tensor) #这个操作必须是初始化之后才能使用
print(tensor_y)

tensor([[0.5418, 0.1970, 0.5264],
        [0.8662, 0.7325, 0.1879]]) 

tensor([[5.5418, 5.1970, 5.5264],
        [5.8662, 5.7325, 5.1879]])
tensor([[5.5418, 5.1970, 5.5264],
        [5.8662, 5.7325, 5.1879]])


In [21]:
print(tensor_y)
tensor_y[:,0].fill_(4) #将第一列的元素全部替换成4，inplace操作
print(tensor_y)

tensor([[4.0000, 5.1970, 5.5264],
        [4.0000, 5.7325, 5.1879]])
tensor([[4.0000, 5.1970, 5.5264],
        [4.0000, 5.7325, 5.1879]])


## Tensor与Numpy

### Tensor转Numpy

In [22]:
t = th.ones(5)
print(f"t: {t}")
n = t.numpy()
print(f"n: {n}")

t: tensor([1., 1., 1., 1., 1.])
n: [1. 1. 1. 1. 1.]


In [23]:
t.add_(1)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([2., 2., 2., 2., 2.])
n: [2. 2. 2. 2. 2.]


### Numpy转Tensor

In [24]:
n = np.ones(5)
t = th.from_numpy(n)

In [25]:
np.add(n, 1, out=n)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([2., 2., 2., 2., 2.], dtype=torch.float64)
n: [2. 2. 2. 2. 2.]
